In [1]:
import z_function_list
import os
from dateutil import relativedelta

In [2]:
base,data,outputs = z_function_list.folder_setup()

In [4]:
os.chdir(data)

In [5]:
os.listdir()

['ADU_raw_list_zip_codes.xlsx',
 'ADU_raw_list_zip_codes_2.xlsx',
 'Attrition and Active Sworn Civilian CHRIS 7-31-2018 (1).xlsx',
 'clean_comparison.xlsx',
 'clean_stat_full.csv',
 'clean_stat_missing_CD.csv',
 'clean_stat_segment_new_project.csv',
 'los_angeles_city_staff.csv',
 'PAYSR Attrition with Years of Service from Pay Period 1 to 7, FY2018-2019.xlsx',
 'Streets_Index_Q1_2016_toQ1_2018',
 'street_scores_with_length.xls',
 'street_scores_with_length.xlsx']

In [9]:
chris_file = pd.read_excel('Attrition and Active Sworn Civilian CHRIS 7-31-2018 (1).xlsx',skiprows=3)
chris_file = chris_file[chris_file['CITY_DEPT']== 70010].copy()
paysr_attrition = pd.read_excel('PAYSR Attrition with Years of Service from Pay Period 1 to 7, FY2018-2019.xlsx',skiprows=1)

In [56]:
attrited_chris = chris_file[chris_file['HIST_FLG']=='H'].copy()
attrited_chris_no_missing = attrited_chris.dropna(subset=['LAPD_APPT_DT']).dropna(subset=['HIST_DT'])

In [113]:
def relative_delta(a,b):
    return relativedelta.relativedelta(a,b)

def fiscal_year_convert(date):
    """Converts any date into fiscal year, based on LA City's Fiscal Year which starts July 1st.
    Use this function with .apply().
    Parameters
    -------------------
    date: date to be transformed. This is a dummy parameter because this function should be used with .apply()."""
    if date.month >= 7:
        return "FY %s-%s" % (date.year, date.year + 1)
    else:
        return "FY %s-%s" % (date.year - 1, date.year)

attrited_chris_no_missing['Time in Force']=attrited_chris_no_missing.apply(lambda row: relative_delta(row['HIST_DT'],row['LAPD_APPT_DT']),axis=1)
attrited_chris_no_missing['Years_In_Force'] = attrited_chris_no_missing['Time in Force'].apply(lambda x: x.years)
attrited_chris_no_missing['FY'] = attrited_chris_no_missing['HIST_DT'].apply(fiscal_year_convert)
less_than_3 = attrited_chris_no_missing[attrited_chris_no_missing['Years_In_Force']<3].copy()

paysr_attrition['Leave_Date_FY'] = paysr_attrition['History\nDate'].apply(fiscal_year_convert)

In [137]:
fy_17_18_paysr = paysr_attrition[paysr_attrition['Leave_Date_FY']=='FY 2017-2018'].copy()

In [138]:
fy_17_18_paysr['Time_Relative_Delta'] = fy_17_18_paysr.apply(lambda row: relative_delta(row['History\nDate'],row['Appointed\nDate']),axis=1)
fy_17_18_paysr['Years_In_Force'] = fy_17_18_paysr['Time_Relative_Delta'].apply(lambda x: x.years)

In [146]:
less_than_3_paysr = fy_17_18_paysr[fy_17_18_paysr['Years_In_Force']<3].copy()

In [148]:
less_than_3_paysr

,Pay Period,Dept,Last Name,First Name,Serial,Appointed Date,History Date,Years of Service,Leave_Date_FY,Time_Relative_Delta,Years_In_Force
63,Pay Period 4,4301,COLLINS,EMMA,43565,2018-05-14,2018-06-29,"0 Years, 1 Months, 15 Days",FY 2017-2018,"relativedelta(months=+1, days=+15)",0
65,Pay Period 3,4301,MARMOLEJO,PAMELA,43497,2018-03-19,2018-06-29,"0 Years, 3 Months, 10 Days",FY 2017-2018,"relativedelta(months=+3, days=+10)",0
73,Pay Period 3,4301,OCASIO NEGRON,OMAR,43144,2017-07-10,2018-06-14,"0 Years, 11 Months, 4 Days",FY 2017-2018,"relativedelta(months=+11, days=+4)",0
74,Pay Period 3,4301,QUINN FOUNTANO,TYLER,43507,2018-03-19,2018-06-29,"0 Years, 3 Months, 10 Days",FY 2017-2018,"relativedelta(months=+3, days=+10)",0
79,Pay Period 3,4301,GARCIA,ESTEBAN,43597,2018-06-11,2018-06-13,"0 Years, 0 Months, 2 Days",FY 2017-2018,relativedelta(days=+2),0
80,Pay Period 3,4301,AMAD,SUSANNE,43592,2018-06-11,2018-06-13,"0 Years, 0 Months, 2 Days",FY 2017-2018,relativedelta(days=+2),0
114,Pay Period 1,4301,GARCIA,CRISTINA,43450,2018-02-20,2018-06-12,"0 Years, 3 Months, 23 Days",FY 2017-2018,"relativedelta(months=+3, days=+23)",0
115,Pay Period 1,4301,CERVANTES,SABRINA,43398,2018-01-22,2018-05-15,"0 Years, 3 Months, 23 Days",FY 2017-2018,"relativedelta(months=+3, days=+23)",0
116,Pay Period 1,4301,SANABRIA,DANIEL,43378,2017-12-26,2018-04-06,"0 Years, 3 Months, 11 Days",FY 2017-2018,"relativedelta(months=+3, days=+11)",0
117,Pay Period 1,4301,DIAZ,AMANDA,43087,2017-06-12,2018-05-15,"0 Years, 11 Months, 3 Days",FY 2017-2018,"relativedelta(months=+11, days=+3)",0
